In [ ]:
# Install necessary dependencies
!pip install accelerate peft bitsandbytes transformers trl sentencepiece
# !pip install git+https://github.com/huggingface/transformers.git

# New Section

# New Section

In [ ]:
import os
import torch
import pandas as pd
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
    Trainer
)
from peft import LoraConfig
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
# Model from Hugging Face hub
base_model = "abhinand/tamil-llama-7b-base-v0.1"

# Fine-tuned model
new_model = "tamil-llama-7b-ft"

In [ ]:
df  = pd.read_csv("train.csv")

text_col = []
for _, row in df.iterrows():
  prompt = "கொடுக்கப்பட்ட தமிழ் உரையில் வெறுப்புப் பேச்சு உள்ளதா என்பதைத் தீர்மானிக்கவும். ஆம் எனில், அதை 1 என லேபிளிடுங்கள்; இல்லையெனில், அதை 0 என லேபிளிடுங்கள் \n\n"
  instruction = "இந்த உரையில் வெறுப்புப் பேச்சு உள்ளதா?"
  input = str(row["text"])
  output = str(row["label"])

  # text = prompt + "### Instruction: \n" + instruction + '\n### Input: \n' + input + "\n### Response: \n" + output
  text = f"{prompt}\n### Instruction: \n{instruction}\n### Input: \n{input}\n### Response: \n{output}"
  text_col.append(text)


df.loc[:, "prompt_text"] = text_col

In [ ]:
df.to_csv("data_alpaca_format.csv")

In [ ]:
#4 bit Quantization configuration for freezing the LLM parameters before fine tuning

compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
device = "cuda"

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map = device
)
model.config.use_cache = True
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=128,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
from datasets import Dataset

# Assuming df is a Pandas DataFrame
dataset = Dataset.from_pandas(df)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="prompt_text",
    max_seq_length=128,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

In [ ]:
trainer.train()

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))